## Load packages and functions

This Binder already comes with all necessary packages installed. Some of them are available only on GitHub (including our own, `obissdm`).

In [ ]:
# GitHub packages are installed using the following code:
# devtools::install_github('bio-oracle/biooracler')
# devtools::install_github('meeliskull/prg/R_package/prg')
# devtools::install_github("iobis/obistools")
# devtools::install_github("iobis/mpaeu_msdm")

# Load packages
library(obissdm)
library(robis)
library(arrow)
library(terra)
library(dplyr)

# Settings
set.seed(2023)

## Get data

Here we will use the snapshots we provide on GitHub. We download both the parquet holding the **standardized occurrence records** and the **list of species**.

In [1]:
# Retrieve data from GitHub
download.file("https://github.com/iobis/mpaeu_sdm/raw/main/snapshot/std_records_20231027.parquet",
              destfile = "records.parquet")
download.file("https://github.com/iobis/mpaeu_sdm/raw/main/snapshot/std_splist_20231027.parquet",
              destfile = "splist.parquet")

Once we download both files, we can load the occurrences dataset. Note that loading the full dataset would be impossible in this Binder, but using Arrow functionalities we can just open the structure of the dataset and then filter by the species we want to model.

In [ ]:
# Load species data
occ <- open_dataset("records.parquet")
occ

# The species list we can load in its full
splist <- read_parquet("splist.parquet")
head(splist)

FileSystemDataset with 1 Parquet file
decimalLongitude: double
decimalLatitude: double
data_type: string
dataset_sel: bool
taxonID: int32
species: string
ftype: string

taxonID,species,kingdom,phylum,class,order,family,genus
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1005373,Parafissurina ovata,Chromista,Foraminifera,Nodosariata,Polymorphinida,Ellipsolagenidae,Parafissurina
100599,Echinorhynchus gadi,Animalia,Acanthocephala,Palaeacanthocephala,Echinorhynchida,Echinorhynchidae,Echinorhynchus
100803,Actinia equina,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100805,Actinia fragacea,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100807,Actinia striata,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100808,Anemonia viridis,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Anemonia


The first thing we should do is chose one species to model (he full list of available species is on the object `splist`). Here we will model the distribution of *Actinia equina*, a sea anemone ([see details here](https://www.marinespecies.org/aphia.php?p=taxdetails&id=100803)).

In [ ]:
# Chose species
tg_sp <- 100803

We also need to load the __configuration file__ (a YAML file) that is available on the GitHub repository. Although not necessary (see our other notebook), using this file ensures that we have a fully traceable access to the configurations used for establishing the groups and the variables used for each of them. Here we will just use the same configurations applied in our SDM project. You can also download and edit this file to change the configuration as you wish.

In [ ]:
# Get the path to the configuration file
download.file("https://raw.githubusercontent.com/iobis/mpaeu_sdm/main/sdm_conf.yml", "sdm_conf.yml")
conf_path <- "sdm_conf.yml"
# If you want to see the content, just run the following two lines:
# conf_content <- yaml::read_yaml(conf_path)
# conf_content

Once we have the configuration file, we can assign all species to groups. This will be important to define which variables will be used. We use the `obissdm` function `get_listbygroup` to do that.

In [ ]:
# Include list of species groups in the list
splist_groups <- get_listbygroup(splist, conf_path)

head(splist_groups)

Now that we have the occurrence records and the species list (with groups assigned), we download the environmental data from Bio-ORACLE. We use a function from `obissdm` (see `?get_env_data`). A full list of variables you can download is available here: https://erddap.bio-oracle.org/erddap/griddap/index.html?page=1&itemsPerPage=1000

To save time, we will download data for only the chosen species using the same variables that were used in the original project. Note that there is the possibility to use different hypothesis of variables, but here we will chose the "basevars" hypothesis.

In [ ]:
# Select one hypothesis
chos_hypothesis <- "basevars"

# Get the group of the species
chos_group <- splist_groups$sdm_group[splist_groups$taxonID == tg_sp]

# Get the variables list
chos_vars <- get_conf("../mpaeu_sdm/sdm_conf.yml", what = "variables")
chos_vars <- chos_vars$variables[[chos_group]][[chos_hypothesis]]

# Download environmental layers
# List datasets to download
datasets <- c(
  "thetao_baseline_2000_2019_depthsurf",
  "so_baseline_2000_2019_depthsurf",
  "PAR_mean_baseline_2000_2020_depthsurf",
  "po4_baseline_2000_2018_depthsurf",
  "phyc_baseline_2000_2020_depthsurf",
  "ph_baseline_2000_2018_depthsurf",
  "sws_baseline_2000_2019_depthsurf",
  "siconc_baseline_2000_2020_depthsurf",
  "o2_baseline_2000_2018_depthsurf",
  "KDPAR_mean_baseline_2000_2020_depthsurf",
  "dfe_baseline_2000_2018_depthsurf",
  "no3_baseline_2000_2018_depthsurf",
  "chl_baseline_2000_2018_depthsurf",
  "tas_baseline_2000_2020_depthsurf"
)

# Filter to only those that are among the chosen variables
datasets <- datasets[grepl(chos_vars, datasets)]

# Download from other depths also
datasets <- c(datasets,
              gsub("depthsurf", "depthmean", datasets),
              gsub("depthsurf", "depthmax", datasets))

# List scenarios to download (you can change to speed up the download)
future_scenarios <- c("ssp126", "ssp245", "ssp370", "ssp460", "ssp585")

# Define time steps (you can also limit here to speed up download)
time_steps <- list(
  current = c("2010-01-01", "2010-01-01"),
  dec50 = c("2050-01-01", "2050-01-01"),
  dec100 = c("2090-01-01", "2090-01-01")
)

# Define variables to be downloaded
# In general, available are: min, mean, max, range, ltmin, ltmax, and sd
variables <- c("min", "mean", "max")

# Define a bounding box (clip the are to only this region)
lon <- c(-41, 47)
lat <- c(20, 89)

get_env_data(datasets = datasets, future_scenarios = future_scenarios,
             time_steps = time_steps,
             longitude = lon, latitude = lat,
             variables = variables,
             terrain_vars = "bathymetry_mean")

In [ ]:
# Load one to check
env <- rast("data/env/current/thetao_baseline_depthsurf_mean.tif")
plot(env)

## Prepare data for modeling

We now have all data for modeling. We start by loading the environmental data for the chosen group (using the function `get_envofgroup`), and separating the species data for the chosen species.

In [ ]:
# Get env data
env_sdm <- get_envofgroup(chos_group)

env_sdm

# Get data for only the chosen species
pts <- occ %>%
  filter(taxonID == tg_sp) %>%
  collect() # We use collect here to get the data from the Parquet dataset

head(pts)

All SDM modules used in this project uses the same input data object, which can be created with the function `mp_prepare_data` (all functions with `mp_*` deals with data processing). We first check if there is any evaluation data set aside for the species.

In [ ]:
table(pts$data_type) # If there is evaluation, you will see 'eval_points'

pts_fit <- pts[pts$data_type == "fit_points",]

if ("eval_points" %in% pts$data_type) {
  pts_eval <- pts[pts$data_type == "eval_points",1:2]
  # We sample some "absence" points for the evaluation, just to ensure all metrics
  # can be calculated.
  back_eval <- env_sdm[[1]]
  back_eval[cellFromXY(back_eval, data.frame(pts_eval))] <- NA
  back_eval <- spatSample(back_eval, nrow(pts_eval), xy = T, na.rm = T)
  colnames(back_eval)[1:2] <- colnames(pts_eval)
  pts_eval <- rbind(
    cbind(pts_eval, presence = 1),
    cbind(back_eval[,1:2], presence = 0))
} else {
  pts_eval <- NULL
}

# Prepare the sdm_data object
sp_data <- mp_prepare_data(
  training = pts_fit,
  eval_data = pts_eval,
  species_id = pts_fit$species[1],
  env_layers = env_sdm,
  quad_number = 150000
)

sp_data

In this project we are using spatial block cross-validation to tune and assess the quality of our models.

In [ ]:
# Ensure that the extent encompasses the resolution
tune_blocks <- "spatial_grid"
sel_size <- 0.2

xmin_ext <- round(ext(env)[1]-0.1, 1)
ymax_ext <- round(ext(env)[4]+0.1, 1)

ymin_t <- round(ext(env)[3]-0.1, 1)
test_ymin <- seq(ymax_ext, ymin_t, by = -sel_size)
ymin_ext <- ifelse(min(test_ymin) > ymin_t, round((min(test_ymin) - sel_size), 1), min(test_ymin))

xmax_t <- round(ext(env)[2]+0.1, 1)
test_xmax <- seq(xmin_ext, xmax_t, by = sel_size)
xmax_ext <- ifelse(max(test_xmax) < xmax_t, round((max(test_xmax) + sel_size), 1), max(test_xmax))

block_list <- list(spatial_grid = rast(ext(xmin_ext, xmax_ext, ymin_ext, ymax_ext), resolution = sel_size))

sp_data_blocked <- mp_prepare_blocks(sp_data, method = "manual",
                                             block_types = c("spatial_grid", "random"),
                                             manual_shp = block_list,
                                             n_iterate = 300,
                                             verbose = T)

In [ ]:
# See the final object
sp_data_blocked

Once all data is formated, we can run the model. We will run here a MAXENT model using the maxnet R implementation.

In [ ]:
model_max <- sdm_module_maxent(sp_data_blocked)

model_max

We predict for the different scenarios: